In [2]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np

In [3]:
RANDOM_STATE = 42

In [4]:
dataset = load_boston()
X = pd.DataFrame(dataset.data)
X.columns = dataset.feature_names
y = dataset.target

1. Разделите выборку на обучающую и тестовую в отношении 80% / 20%.

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((404, 13), (102, 13), (404,), (102,))

2. Обучите стандартную регрессию, а также Ridge и  Lasso c параметрами по умолчанию и выведите их R2 на тестовой выборке.

In [7]:
from sklearn.linear_model import Ridge, Lasso, LinearRegression

In [8]:
names = ['Ridge', 'Lasso', 'Vanilla Linear']
models = [Ridge(), Lasso(), LinearRegression()]
for name, model in zip(names, models):
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    print(f'R^2 of {name} model: {score:.4f}')

R^2 of Ridge model: 0.6662
R^2 of Lasso model: 0.6671
R^2 of Vanilla Linear model: 0.6688


3. Для Ridge и Lasso подберите коэффициент регуляризации (используйте GridSearchCV, RidgeCV, LassoCV) в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по лучшим моделям и сравните с предыдущими результатами. Напишите как изменился результат.

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RidgeCV, LassoCV

In [10]:
params = {'alpha': [10**n for n in range(-5, 6)]}
cv_models = []
cv_models.append(GridSearchCV(estimator=Ridge(), param_grid=params))
cv_models.append(GridSearchCV(estimator=Lasso(), param_grid=params))
cv_models.append(RidgeCV(alphas=params['alpha']))
cv_models.append(LassoCV(alphas=params['alpha']))

In [11]:
cv_names = ['Grid Search Ridge', 'Grid Search Lasso', 'Ridge CV', 'Lasso CV']
for name, model in zip(cv_names, cv_models):
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    print(f'R^2 of {name} model: {score:.4f}')

R^2 of Grid Search Ridge model: 0.6688
R^2 of Grid Search Lasso model: 0.6688
R^2 of Ridge CV model: 0.6688
R^2 of Lasso CV model: 0.6688


### Вывод:
R^2 не поменялся от регуляризации и остался на уровне обычной линейной регрессии

4. Проведите масштабирование выборки (используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [13]:
models = [Ridge(), Lasso(), LinearRegression()]
m_names = ['ridge', 'lasso', 'vanilla_linear']
scalers = [StandardScaler(), MinMaxScaler()]
s_names = ['standard', 'min_max']
for model in zip(m_names, models):
    for scaler in zip(s_names, scalers):
        pipe = Pipeline(steps=[scaler, model])
        pipe.fit(X_train, y_train)
        score = pipe.score(X_test, y_test)
        print(f'R^2 of {model[0]} model with {scaler[0]} scaler: {score:.4f}')

R^2 of ridge model with standard scaler: 0.6685
R^2 of ridge model with min_max scaler: 0.6764
R^2 of lasso model with standard scaler: 0.6239
R^2 of lasso model with min_max scaler: 0.2574
R^2 of vanilla_linear model with standard scaler: 0.6688
R^2 of vanilla_linear model with min_max scaler: 0.6688


### Вывод:
Модели со стандартными настройками кроме Lasso показали примерно такой же результат.
Lasso показал очень плохой результат с Min Max масштабированием.

5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [14]:
models = [RidgeCV(alphas=params['alpha']), LassoCV(alphas=params['alpha'])]
m_names = ['ridge_cv', 'lasso_cv']
scalers = [StandardScaler(), MinMaxScaler()]
s_names = ['standard', 'min_max']
for model in zip(m_names, models):
    for scaler in zip(s_names, scalers):
        pipe = Pipeline(steps=[scaler, model])
        pipe.fit(X_train, y_train)
        score = pipe.score(X_test, y_test)
        print(f'R^2 of {model[0]} model with {scaler[0]} scaler: {score:.4f}')

R^2 of ridge_cv model with standard scaler: 0.6660
R^2 of ridge_cv model with min_max scaler: 0.6700
R^2 of lasso_cv model with standard scaler: 0.6688
R^2 of lasso_cv model with min_max scaler: 0.6688


### Вывод:
Благодаря регулиризации R^2 вернулся в исходный уровень в 0.66-0.67

6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [15]:
from sklearn.preprocessing import PolynomialFeatures
import warnings
warnings.filterwarnings('ignore')

In [16]:
models = [RidgeCV(alphas=params['alpha']), LassoCV(alphas=params['alpha'])]
m_names = ['ridge_cv', 'lasso_cv']
scalers = [StandardScaler(), MinMaxScaler()]
s_names = ['standard', 'min_max']
for model in zip(m_names, models):
    for scaler in zip(s_names, scalers):
        pipe = Pipeline(steps=[scaler, ('pol_feat', PolynomialFeatures(include_bias=False)), model])
        pipe.fit(X_train, y_train)
        score = pipe.score(X_test, y_test)
        print(f'R^2 of {model[0]} model with {scaler[0]} scaler : {score:.4f}')

R^2 of ridge_cv model with standard scaler : 0.8180
R^2 of ridge_cv model with min_max scaler : 0.8501
R^2 of lasso_cv model with standard scaler : 0.8122
R^2 of lasso_cv model with min_max scaler : 0.8391


### Вывод:
Использование попарных произведений признаков значительно увеличило коэффициент детерминации до максимума 0.85 для Ridge с масштабированием min_max

7. Подберите наилучшую модель (используйте Pipeline, GridSearchCV) подбирая тип регуляризации (L1, L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2. Напишите как изменился R2 по сравнению с предыдущими экспериментами

In [17]:
steps = [('scaler', StandardScaler()),
         ('polynomial', PolynomialFeatures(include_bias=False)),
         ('model', Ridge())]
params = {'scaler': [StandardScaler(), MinMaxScaler()],
          'polynomial__degree': [2, 3, 4],
          'model': [Ridge(), Lasso()],
          'model__alpha': params['alpha']
         }
pipe = Pipeline(steps=steps)
search_model = GridSearchCV(pipe, param_grid=params, n_jobs=2)

In [18]:
search_model.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('polynomial',
                                        PolynomialFeatures(include_bias=False)),
                                       ('model', Ridge())]),
             n_jobs=2,
             param_grid={'model': [Ridge(), Lasso(alpha=0.001)],
                         'model__alpha': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1,
                                          10, 100, 1000, 10000, 100000],
                         'polynomial__degree': [2, 3, 4],
                         'scaler': [StandardScaler(), MinMaxScaler()]})

In [19]:
score = search_model.score(X_test, y_test)
best_params = search_model.best_params_

In [20]:
from pprint import pprint
print('Parameters of the best model:')
pprint(best_params)
print()
print(f'R^2 of the best model: {score:.4f}')

Parameters of the best model:
{'model': Lasso(alpha=0.001),
 'model__alpha': 0.001,
 'polynomial__degree': 3,
 'scaler': MinMaxScaler()}

R^2 of the best model: 0.8450


### Вывод:
Как видим лучшая модель имеет L1 регуляризацию с парметром $\alpha = 0.001$, степень полинома 3 и метод масштабирования Min Max.
Однако на тестовых данных модель показала чуть меньший коэффициент детерминации, чем модель L2 на масштабировании Min Max и степени полинома 2. (0.845 против 0.850 соответственно)

http://archive.ics.uci.edu/ml/datasets/Adult<br>
https://raw.githubusercontent.com/jbrownlee/Datasets/master/adult-all.csv

age: continuous.<br>
workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.<br>
fnlwgt: continuous.<br>
education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.<br>
education-num: continuous.<br>
marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.<br>
occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.<br>
relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.<br>
race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.<br>
sex: Female, Male.<br>
capital-gain: continuous.<br>
capital-loss: continuous.<br>
hours-per-week: continuous.<br>
native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

In [21]:
columns = ['age',
           'workclass',
           'fnlwgt',
           'education',
           'education-num',
           'marital-status',
           'occupation',
           'relationship',
           'race',
           'sex',
           'capital-gain',
           'capital-loss',
           'hours-per-week',
           'native-country',
           'year-salary'
          ]

In [22]:
data = pd.read_csv('adult-all.csv', header=None)
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num',
    'marital-status', 'occupation', 'relationship', 'race', 'sex',
    'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
    'yearly-salary'
]

data.columns = columns
data.tail()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,yearly-salary
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K
48838,64,?,321403,HS-grad,9,Widowed,?,Other-relative,Black,Male,0,0,40,United-States,<=50K
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K
48841,35,Self-emp-inc,182148,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,60,United-States,>50K


8. Разделите выборку на признаки и целевую переменную(колонка со зачениями {<=50K,>50K}). Замените целевую переменную на числовые значения.

In [23]:
X, y = data.iloc[:, :-1], data.iloc[:, -1]
y = pd.Series(np.where(y == '<=50K', 1, -1))
y.value_counts()
X.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
dtype: object

9. Выясните, присутствуют ли в данных пропуски. Заполните их самыми частыми значениями (используйте SimpleImputer)

In [26]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       48842 non-null  object
 2   fnlwgt          48842 non-null  int64 
 3   education       48842 non-null  object
 4   education-num   48842 non-null  int64 
 5   marital-status  48842 non-null  object
 6   occupation      48842 non-null  object
 7   relationship    48842 non-null  object
 8   race            48842 non-null  object
 9   sex             48842 non-null  object
 10  capital-gain    48842 non-null  int64 
 11  capital-loss    48842 non-null  int64 
 12  hours-per-week  48842 non-null  int64 
 13  native-country  48842 non-null  object
dtypes: int64(6), object(8)
memory usage: 5.2+ MB


In [27]:
y.isnull().any()

False

### Вывод:
В данных нет пропусков вида null

10. Выберите колонки с числовыми и категориальными переменными.

In [32]:
cat_cols, num_cols = [], []
for name, tp in X.dtypes.to_dict().items():
    if tp == 'int64':
        num_cols.append(name)
    else:
        cat_cols.append(name)

print('categorical:', cat_cols)
print('numerical:', num_cols)

categorical: ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
numerical: ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']


11. Создайте пайплайн по обработке колонок (используйте OneHotEncoder, MinMaxScaler).

In [29]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [30]:
cat_trans = OneHotEncoder(handle_unknown='ignore')
num_trans = MinMaxScaler()
trans = ColumnTransformer([('num', num_trans, num_cols),
                           ('cat', cat_trans, cat_cols)])

12. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [31]:
accuracy = max(y.value_counts()) / sum(y.value_counts())
print(f'Accuracy: {accuracy:.4f}')
TP = max(y.value_counts())
FP = min(y.value_counts())
TN = min(y.value_counts())
FN = 0
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1 = 2 * precision * recall / (precision + recall)
print(f'f1 score: {f1:.4f}')

Accuracy: 0.7607
f1 score: 0.8641


13. Посчитайте cross_val_score по алгоритмам LogisticRegression, SVC, LinearSVC по метрикам accuracy и f1_score.
Напишите удалось ли превзойти предыдущий результат.

In [36]:
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score, f1_score

In [38]:
models = [
    LogisticRegression(random_state=RANDOM_STATE, max_iter=10000),
    SVC(random_state=RANDOM_STATE),
    LinearSVC(random_state=RANDOM_STATE)
]

for model in models:
    pipe = Pipeline(steps=[('trans', trans), ('model', model)])
    print('Model:', model)
    scores = cross_validate(pipe, X, y, scoring=('accuracy', 'f1'))
    print('accuracy:', np.mean(scores['test_accuracy']))
    print('f1 score:', np.mean(scores['test_f1']))

Model: LogisticRegression(max_iter=10000, random_state=42)
accuracy: 0.8509071111051867
f1 score: 0.904835410336237
Model: SVC(random_state=42)
accuracy: 0.839994419828589
f1 score: 0.8986477856490989
Model: LinearSVC(random_state=42)
accuracy: 0.8529135478362626
f1 score: 0.9063223083526415


### Вывод
Результат превзойден по всем моделям примерно на 0.05 как по accuracy, так и по f1

14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями (испольуйте SimpleImputer)

In [42]:
from sklearn.impute import SimpleImputer

In [40]:
null_killer = SimpleImputer(missing_values='?', strategy='most_frequent')
newX = pd.DataFrame(null_killer.fit_transform(X),
                    columns=X.columns).astype(X.dtypes.to_dict())

15. Посчитайте cross_val_score на новых данных. Напишите удалось ли улучшить результат.

In [41]:
models = [
    LogisticRegression(random_state=RANDOM_STATE, max_iter=10000),
    SVC(random_state=RANDOM_STATE),
    LinearSVC(random_state=RANDOM_STATE)
]

for model in models:
    pipe = Pipeline(steps=[('trans', trans), ('model', model)])
    print(model)
    scores = cross_validate(pipe, newX, y, scoring=('accuracy', 'f1'))
    print('accuracy:', np.mean(scores['test_accuracy']))
    print('f1 score:', np.mean(scores['test_f1']))

LogisticRegression(max_iter=10000, random_state=42)
accuracy: 0.8504566501751475
f1 score: 0.9046445905552222
SVC(random_state=42)
accuracy: 0.8395848963639935
f1 score: 0.8985494397611463
LinearSVC(random_state=42)
accuracy: 0.8508251523375897
f1 score: 0.905141981677368


### Вывод
Результат практически не поменялся

16. Посчитайте cross_val_score, если просто удалить значения '?'. Напишите как изменился результат

In [43]:
nonnull_rows = (X != '?').all(axis=1)
dropX = X[nonnull_rows]
dropy = y[nonnull_rows]

models = [
    LogisticRegression(random_state=RANDOM_STATE, max_iter=10000),
    SVC(random_state=RANDOM_STATE),
    LinearSVC(random_state=RANDOM_STATE)
]

for model in models:
    pipe = Pipeline(steps=[('trans', trans), ('model', model)])
    print('Model:', model)
    scores = cross_validate(pipe, dropX, dropy, scoring=('accuracy', 'f1'))
    print('accuracy:', np.mean(scores['test_accuracy']))
    print('f1 score:', np.mean(scores['test_f1']))

Model: LogisticRegression(max_iter=10000, random_state=42)
accuracy: 0.8468445672761554
f1 score: 0.901151790779792
Model: SVC(random_state=42)
accuracy: 0.8356995943179577
f1 score: 0.8946665218391704
Model: LinearSVC(random_state=42)
accuracy: 0.8485030154158197
f1 score: 0.9024033119235104


### Вывод
Результат слегка ухудшился (3 знак после запятой)

 17. Посчитайте cross_val_score для RandomForestClassifier, GradientBoostingClassifier. Напишите как изменился результат и какой вывод можно из этого сделать.

In [44]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

In [46]:
models = [
    RandomForestClassifier(random_state=RANDOM_STATE),
    GradientBoostingClassifier(random_state=RANDOM_STATE)
]

for model in models:
    pipe = Pipeline(steps=[('trans', trans), ('model', model)])
    print('Model:', model)
    scores = cross_validate(pipe, dropX, dropy, scoring=('accuracy', 'f1'))
    print('accuracy:', np.mean(scores['test_accuracy']))
    print('f1 score:', np.mean(scores['test_f1']))

Model: RandomForestClassifier(random_state=42)
accuracy: 0.8481491701158076
f1 score: 0.9010790294674113
Model: GradientBoostingClassifier(random_state=42)
accuracy: 0.8629429514670492
f1 score: 0.9122547730675745


### Вывод
На текущий момент лучшая модель - это Градиентный Бустинг.<br>
Случайный лес примерно такого же качества что и предыдущие модели.

18. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [48]:
from sklearn.preprocessing import OrdinalEncoder

In [59]:
encoders = [
    OneHotEncoder(handle_unknown='ignore'),
    OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
]
scalers = [MinMaxScaler(), StandardScaler()]
models = [
    RandomForestClassifier(random_state=RANDOM_STATE),
    GradientBoostingClassifier(random_state=RANDOM_STATE),
    LogisticRegression(random_state=RANDOM_STATE, max_iter=10000),
    # SVC(random_state=RANDOM_STATE), - зависает на 2 часа
    LinearSVC(random_state=RANDOM_STATE)
]

imp = SimpleImputer(missing_values='?')
cat_trans = Pipeline([('imputer', imp), ('encoder', OneHotEncoder())])
trans = ColumnTransformer([('num', MinMaxScaler(), num_cols),
                           ('cat', cat_trans, cat_cols)])
pipe = Pipeline(steps=[('trans', trans), ('model', SVC())])
params = {
    'trans__num': scalers,
    'trans__cat__encoder': encoders,
    'trans__cat__imputer__strategy': ['most_frequent', 'constant'],
    'model': models
}

search = GridSearchCV(pipe,
                      param_grid=params,
                      scoring=('accuracy', 'f1'),
                      refit='f1')

In [60]:
search.fit(X, y)

GridSearchCV(estimator=Pipeline(steps=[('trans',
                                        ColumnTransformer(transformers=[('num',
                                                                         MinMaxScaler(),
                                                                         ['age',
                                                                          'fnlwgt',
                                                                          'education-num',
                                                                          'capital-gain',
                                                                          'capital-loss',
                                                                          'hours-per-week']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                       

In [66]:
print('Итоговые параметры модели:')
search.best_params_

Итоговые параметры модели:


{'model': GradientBoostingClassifier(random_state=42),
 'trans__cat__encoder': OneHotEncoder(handle_unknown='ignore'),
 'trans__cat__imputer__strategy': 'constant',
 'trans__num': MinMaxScaler()}

In [67]:
res = pd.DataFrame(search.cv_results_)
print('Accuracy и f1:')
res[['mean_test_accuracy',
     'mean_test_f1']][res['mean_test_f1'] == max(res['mean_test_f1'])].apply(max)

Accuracy и f1:


mean_test_accuracy    0.867573
mean_test_f1          0.916026
dtype: float64